In [ ]:
from agent import agent
from board import board
from moves import *
from model import Net
import torch
import copy
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def generate_trajectory(actor,critic, max_t=100):
        Agent=agent(copy.deepcopy(board))

        # critic=Critic()
        saved_log_probs = []
        rewards = []
        state_values=[]
        # print("generate trajectory")
        
        

        for t in range(max_t):
            Agent.visualize()
            
            state=Agent.board_to_array()
            mask,mp=Agent.rel_to_abs_moves()
            #state_val= critic(state)
            # try:
            action_id, log_prob =  actor.sample(state,mask)
            state_value=critic(state)
            state_values.append(state_value)
            # except:
            #      print('no')
            #      break
                
            action=mp[action_id]





            white_reward= Agent.make_move(action['tag'],action['to'],action['castles'])

            saved_log_probs.append(log_prob)
            if Agent.checkmate("B"):
                rewards.append(30)
                print("white won")
                break
            if Agent.stalemate("B"):
                
                rewards.append(-2)
                break

            # add te obtained results to their relative lists ==> saved_log_probs, rewards, state_values
            tag,to,castles=Agent.sample("B")
            
            black_reward=Agent.make_move(tag,to,castles)
            
            if Agent.checkmate("W"):
                rewards.append(-30)
                print("black won")
                break
            if Agent.stalemate("W"):
                
                rewards.append(2)
                break

            # add code here
            
            # add code here
            rewards.append(white_reward - black_reward)
           
            # add code here

        # Agent.visualize()
        return  saved_log_probs , rewards , state_values



In [ ]:
# PyTorch
import torch
import torch.nn as nn

from torch.distributions import Categorical

#Using a neural network to learn state value
class Critic(nn.Module):
        def __init__(self):
            super(Critic, self).__init__()
            self.conv1 = nn.Conv2d(6, 32, kernel_size=3, stride=1, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
            self.fc1 = nn.Linear(256, 128)
            self.fc2 = nn.Linear(128, 64)
            self.fc3 = nn.Linear( 64, 1)
            self.flatten=nn.Flatten()

        def forward(self, x):
            x = torch.from_numpy(x).float().unsqueeze(0)
            x = self.pool(self.conv1(x))
            x = self.pool(self.conv2(x))
            x = self.flatten(x)
            x = self.fc1(x)
            x=self.fc2(x)
            return self.fc3(x) 


In [ ]:
from collections import deque
def computer_cumulative_reward(rewards, max_t,gamma):
        returns = deque(maxlen=max_t)
        n_steps = len(rewards)
        for t in range(n_steps)[::-1]:
          disc_return_t = (returns[0] if len(returns)>0 else 0)
          returns.appendleft( rewards[t]+gamma*disc_return_t)
        return returns

In [ ]:
import numpy as np
def returns_standardization(returns):
        eps = np.finfo(np.float32).eps.item()
        ## eps is the smallest representable float, which is
        # added to the standard deviation of the returns to avoid numerical instabilities
        returns = torch.tensor(returns)
        returns = (returns - returns.mean()) / (returns.std() + eps)

        return returns

In [ ]:
def train(optimizer,saved_log_probs, returns , state_values):
        
        state_values= torch.stack(state_values).squeeze()
        advantages = returns - state_values
        advantages = [g - val for g, val in zip(returns,state_values)]
        advantages = torch.Tensor(advantages).to(device)
        losses = []
        for log_prob, advantage in zip(saved_log_probs, returns):
          losses.append(-log_prob *   advantage)
        loss = torch.sum(torch.stack(losses))
        print(loss)

        # actor_loss = torch.cat(loss).sum()
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
def train_critic(optimizer,rewards,pred):
    rewards=torch.tensor(rewards,requires_grad=True)
    pred=torch.tensor(pred,requires_grad=True)
    loss = F.mse_loss(pred, rewards)  # Mean Squared Error (MSE) Loss
    print(loss)
    optimizer.zero_grad()  # Clear the gradients
    loss.backward()  # Compute the backward pass
    optimizer.step() 
    return loss    # Update the weights based on the computed gradients</s>

In [ ]:
def critic_fn(actor,critic,actor_optimizer,critic_optimizer, n_training_episodes, max_t, gamma, print_every=1):
    # Help us to calculate the score during the training
    scores_deque = deque(maxlen=100)
    scores = []
    losses=[]

    for i_episode in range(1, n_training_episodes+1):

        # Generate an episode
        #add code here
        log_probs, rewards,states = generate_trajectory(actor,critic, max_t)
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        # calculate the return
        returns= computer_cumulative_reward(rewards,max_t,gamma)
        

        ## standardization of the returns is employed to make training more stable
        returns=returns_standardization(returns)
        # print(returns)

        # Train the Critic network
        #add code here
        loss=train_critic(critic_optimizer, returns,states)
        train(actor_optimizer,log_probs,returns,states)
        losses.append(loss)





        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))

    return scores

In [ ]:
def Actor(actor,optimizer, n_training_episodes, max_t, gamma, print_every=1):
    # Help us to calculate the score during the training
    scores_deque = deque(maxlen=100)
    scores = []

    for i_episode in range(1, n_training_episodes+1):

        # Generate an episode
        #add code here
        saved_log_probs, rewards = generate_trajectory(actor, max_t)
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        # calculate the return
        returns= computer_cumulative_reward(rewards,max_t,gamma)
        

        ## standardization of the returns is employed to make training more stable
        returns=returns_standardization(returns)
        print(returns)

        # Train the Critic network
        #add code here
        train(optimizer,saved_log_probs, returns)





        # if i_episode % print_every == 0:
        #     print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))

    return scores

In [ ]:
Agent=agent(board)
arr=Agent.board_to_array()
actor.sample(arr)

In [ ]:
# a=torch.tensor([0,1,0,0,0]).float()
# sampled_indices = torch.multinomial(a,1)
# sampled_indices[0].item()

In [ ]:
actor=Net()
critic=Critic()
actor_optimizer = torch.optim.Adam(actor.parameters())
critic_optimizer = torch.optim.Adam(critic.parameters())


In [ ]:
n_training_episodes= 500
max_t = 50
gamma = 0.75

In [ ]:
loss_history=critic_fn(actor,critic,actor_optimizer,critic_optimizer,n_training_episodes,max_t,gamma)

In [ ]:
torch.save(critic.state_dict(),'critic.pth')
torch.save(actor.state_dict(),'actor.pth')
# scores=Actor(model , optimizer , n_training_episodes , max_t , gamma )

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
scores= pd.Series(loss_history, name="scores_Actor")
scores.describe()



In [ ]:
fig, ax = plt.subplots(1, 1)
_ = scores.plot(ax=ax, label="scores_Actor")
_ = (scores.rolling(window=100)
           .mean()
           .rename("Rolling Average")
           .plot(ax=ax))
ax.legend()
_ = ax.set_xlabel("Episode Number")
_ = ax.set_ylabel("scores_Actor")

In [ ]:
Agent=agent(board)
generate_trajectory(actor,critic,50)

In [ ]:
list('aaaa')